# Simple RAG (Retrieval-Augmented Generation) System

## Descripción general

Este código implementa un sistema básico de Generación Aumentada por Recuperación (RAG) para procesar y consultar documentos PDF. El sistema codifica el contenido del documento en un almacén vectorial, que luego puede ser consultado para recuperar información relevante.

**Componentes Clave**

1. Procesamiento y extracción de texto de PDF
2. Fragmentación del texto para un procesamiento manejable
3. Creación de almacén vectorial utilizando FAISS y embeddings de OpenAI
4. Configuración del recuperador para consultar los documentos procesados
5. Evaluación del sistema RAG

## Detalles del Método

**Preprocesamiento del Documento**

El PDF se carga utilizando PyPDFLoader.
El texto se divide en fragmentos utilizando RecursiveCharacterTextSplitter con un tamaño y superposición especificados.

**Limpieza del Texto**

Se aplica una función personalizada `replace_t_with_space` para limpiar los fragmentos de texto. Esto probablemente aborda problemas específicos de formato en el PDF.

**Creación del Almacén Vectorial**

Se utilizan embeddings de OpenAI para crear representaciones vectoriales de los fragmentos de texto.
Se crea un almacén vectorial FAISS a partir de estos embeddings para una búsqueda de similitud eficiente.

**Configuración del Recuperador**

Se configura un recuperador para obtener los 2 fragmentos más relevantes para una consulta dada.

**Función de Codificación**

La función `encode_pdf` encapsula todo el proceso de carga, fragmentación, limpieza y codificación del PDF en un almacén vectorial.

**Características Clave**

1. Diseño Modular: El proceso de codificación está encapsulado en una sola función para facilitar su reutilización.
2. Fragmentación Configurable: Permite ajustar el tamaño y la superposición de los fragmentos.
3. Recuperación Eficiente: Utiliza FAISS para una búsqueda de similitud rápida.
4. Evaluación: Incluye una función para evaluar el rendimiento del sistema RAG.

**Ejemplo de Uso**

El código incluye una consulta de prueba: "¿Cuál es la principal causa del cambio climático?". Esto demuestra cómo utilizar el recuperador para obtener contexto relevante del documento procesado.

**Evaluación**

El sistema incluye una función `evaluate_rag` para evaluar el rendimiento del recuperador, aunque las métricas específicas utilizadas no se detallan en el código proporcionado.

**Beneficios de este Enfoque**

1. Escalabilidad: Puede manejar documentos grandes procesándolos en fragmentos.
2. Flexibilidad: Fácil de ajustar parámetros como el tamaño del fragmento y el número de resultados recuperados.
3. Eficiencia: Utiliza FAISS para una búsqueda de similitud rápida en espacios de alta dimensión.
4. Integración con NLP Avanzado: Utiliza embeddings de OpenAI para una representación de texto de vanguardia.

**Conclusión**

Este simple sistema RAG proporciona una base sólida para construir sistemas más complejos de recuperación de información y respuesta a preguntas. Al codificar el contenido del documento en un almacén vectorial, permite la recuperación eficiente de información relevante en respuesta a consultas. Este enfoque es particularmente útil para aplicaciones que requieren acceso rápido a información específica dentro de documentos o colecciones de documentos grandes.

### Librerias

1. pypdf

- Propósito: Biblioteca para manipular archivos PDF.

- Uso: Extraer texto, metadatos, dividir o combinar PDFs.

2. PyMuPDF (también conocida como fitz)

- Propósito: Otra biblioteca para trabajar con PDFs, más potente que PyPDF.

- Uso: Extracción avanzada de texto, imágenes y metadatos de PDFs.

3. python-dotenv

- Propósito: Manejo de variables de entorno.

- Uso: Cargar configuraciones sensibles (como claves API) desde archivos .env.

4. langchain-community

- Propósito: Contiene integraciones comunitarias para LangChain.

- Uso: Conectores con múltiples servicios y herramientas de IA.

5. langchain_openai

- Propósito: Integración oficial de LangChain con los modelos de OpenAI.

- Uso: Usar modelos como GPT-3.5/4, embeddings de OpenAI, etc.

6. rank_bm25

- Propósito: Algoritmo de ranking para recuperación de información.

- Uso: Encontrar documentos relevantes basados en términos de búsqueda.

7. faiss-cpu

- Propósito: Biblioteca para búsqueda de similitud vectorial.

- Uso: Búsqueda eficiente de vectores similares (útil para embeddings).

8. deepeval

- Propósito: Evaluación de respuestas de LLMs.

- Uso: Medir la calidad de las respuestas generadas.

In [ ]:
# Install required packages
!pip install pypdf==5.6.0
!pip install PyMuPDF==1.26.1
!pip install python-dotenv==1.1.0
!pip install langchain-community==0.3.25
!pip install langchain_openai==0.3.23
!pip install rank_bm25==0.2.2
!pip install faiss-cpu==1.11.0
!pip install deepeval==3.1.0
!pip install sentence-transformers

In [ ]:
import os
import sys

In [ ]:
import sys
from pathlib import Path

# Sube un nivel en la jerarquía de directorios
sys.path.append(str(Path.cwd().parent))

# Ahora deberías poder importar
from helper_functions import (
    EmbeddingProvider,
    retrieve_context_per_question,
    replace_t_with_space,
    get_langchain_embedding_provider,
    show_context
)
from helper_functions import (EmbeddingProvider,
                              retrieve_context_per_question,
                              replace_t_with_space,
                              get_langchain_embedding_provider,
                              show_context)

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from evaluation.evaluate_rag import evaluate_rag
from langchain.vectorstores import FAISS

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings  # Alternativa local

Leer documento

In [ ]:
import os

# Obtén la ruta absoluta del directorio del proyecto
project_dir = os.path.dirname(os.path.dirname(os.path.abspath(".")))
path = os.path.join(project_dir, "RAG_Techniques", "data", "Understanding_Climate_Change.pdf")

Codificar documento, es decir, convertirlo en un vector de características que capturen su contenido.

In [ ]:
def encode_pdf(
    path: str,
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
    embedding_provider: str = "openai",
    model_name: str = "text-embedding-ada-002"
) -> FAISS:
    """
    Codifica un documento PDF en un almacén vectorial usando embeddings.

    Args:
        path: Ruta al archivo PDF.
        chunk_size: Tamaño de cada fragmento de texto (en tokens).
        chunk_overlap: Superposición entre fragmentos consecutivos.
        embedding_provider: Proveedor de embeddings ("openai" o "huggingface").
        model_name: Nombre del modelo a utilizar para los embeddings.

    Returns:
        FAISS: Almacén vectorial con el contenido del PDF.

    Raises:
        FileNotFoundError: Si el archivo PDF no existe.
        ValueError: Si los parámetros son inválidos.
        Exception: Para otros errores durante el procesamiento.
    """
    import logging
    from pathlib import Path
    import time

    # Configuración de logging
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)

    try:
        # Validación de parámetros
        if not Path(path).is_file():
            raise FileNotFoundError(f"No se encontró el archivo: {path}")
        if chunk_size <= 0 or chunk_overlap < 0:
            raise ValueError("chunk_size debe ser > 0 y chunk_overlap >= 0")
        if chunk_overlap >= chunk_size:
            raise ValueError("chunk_overlap debe ser menor que chunk_size")

        logger.info(f"Cargando documento: {path}")
        start_time = time.time()

        # Cargar PDF
        loader = PyPDFLoader(path)
        documents = loader.load()
        
        if not documents:
            raise ValueError("El documento PDF está vacío o no pudo ser procesado")

        logger.info(f"Documento cargado. Páginas: {len(documents)}")
        logger.info("Dividiendo texto en chunks...")

        # Dividir en chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len
        )
        chunks = text_splitter.split_documents(documents)
        
        if not chunks:
            raise ValueError("No se pudieron generar chunks del documento")

        logger.info(f"Generados {len(chunks)} chunks de texto")

        # Limpiar texto
        logger.info("Limpiando texto...")
        cleaned_chunks = replace_t_with_space(chunks)

        # Seleccionar proveedor de embeddings
        logger.info(f"Usando embeddings de {embedding_provider}...")
        if embedding_provider.lower() == "huggingface":
            from langchain_community.embeddings import HuggingFaceEmbeddings
            embeddings = HuggingFaceEmbeddings(model_name=model_name)
        else:  # Por defecto usa OpenAI
            from langchain_openai import OpenAIEmbeddings
            embeddings = OpenAIEmbeddings(model=model_name)

        # Crear vector store
        logger.info("Creando almacén vectorial...")
        vectorstore = FAISS.from_documents(cleaned_chunks, embeddings)

        elapsed = time.time() - start_time
        logger.info(f"Proceso completado en {elapsed:.2f} segundos")
        
        return vectorstore

    except Exception as e:
        logger.error(f"Error al procesar el documento: {str(e)}")
        raise

In [ ]:
chunks_vector_store = encode_pdf(
    path, 
    chunk_size=1000, 
    chunk_overlap=200,
    embedding_provider="huggingface",
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

Crear un sistema de recuperación y generación (RAG) simple

In [ ]:
# Obtener los 2 fragmentos más relevantes para una consulta dada
chunks_query_retriever = chunks_vector_store.as_retriever(search_kwargs={"k": 2})

Test

In [ ]:
query = "¿Cuál es la principal causa del cambio climático?"
context = retrieve_context_per_question(query, chunks_query_retriever)

show_context(context)

Evaluar resultados de recuperación

In [ ]:
evaluate_rag(chunks_query_retriever)